### Load the model, minimum values & maximum values

In [1]:
import numpy as np
import tensorflow as tf
model = tf.keras.models.load_model("Model/my_model.h5", compile=False)

mins = np.load("Model/mins.npy")
maxs = np.load("Model/maxs.npy")

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 10)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │      5,824 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 8, 8, 512) │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 8, 8,      │  4,719,616 │ max_pooling2d_3[… │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 8, 8,      │  9,438,208 │ conv2d_8[0][0]    │
│                     │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 16, 16,    │          0 │ conv2d_9[0][0]    │
│ (UpSampling2D)      │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16, 16,    │          0 │ up_sampling2d[0]

 Total params: 31,382,977 (119.72 MB)

 Trainable params: 31,382,977 (119.72 MB)

 Non-trainable params: 0 (0.00 B)

### Create FLASK UI

In [2]:
from flask import Flask, request, render_template, url_for
import os
import tifffile as tiff

from PIL import Image

app = Flask("my_app")

app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['RESULT_FOLDER'] = 'static/results'

os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs(app.config['RESULT_FOLDER'], exist_ok=True)

# Load your trained model once
model = tf.keras.models.load_model("Model/my_model.h5", compile=False)

def preprocess_image(img_array, mins, maxs):
    if img_array.ndim == 4:
        img_array = img_array[0]
        
    img_array = np.delete(img_array, [7, 10], axis=-1)
    img_array = (img_array - mins) / (maxs - mins + 1e-8)
    return np.expand_dims(img_array, axis=0)  # add batch dim

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        file = request.files["file"]
        filepath = os.path.join(app.config["UPLOAD_FOLDER"], file.filename)
        file.save(filepath)

        # ---- Load TIFF ----
        img_array = tiff.imread(filepath)

        images_norm = preprocess_image(img_array,mins,maxs)
        
        # ---- Predict ----
        pred_mask = model.predict(images_norm)[0]  # (128,128,1)

        # Convert mask to image
        pred_mask = (pred_mask > 0.5).astype("uint8") * 255
        mask_img = Image.fromarray(pred_mask.squeeze())

        # Save results into static folder
        mask_filename = f"mask_{file.filename}.png"
        result_path = os.path.join(app.config["RESULT_FOLDER"], mask_filename)
        mask_img.save(result_path)

        # Create a quick RGB preview of the input (first 3 channels)
        rgb_array = img_array[..., [3, 2, 1]]
        rgb_array = ((rgb_array - rgb_array.min()) / (rgb_array.max() - rgb_array.min()) * 255).astype(np.uint8)
        
        rgb_preview = Image.fromarray(rgb_array)

        preview_filename = f"preview_{file.filename}.png"
        preview_path = os.path.join(app.config["UPLOAD_FOLDER"], preview_filename)
        rgb_preview.save(preview_path)

        
        return render_template(
            "result.html",
            input_image=url_for("static", filename=f"uploads/{preview_filename}"),
            result_image=url_for("static", filename=f"results/{mask_filename}")
        )

    return render_template("upload.html")


def beginUI():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)


In [3]:
beginUI()

 * Serving Flask app 'my_app'


 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.13:5000
Press CTRL+C to quit
